In [1]:
import pandas as pd 

In [2]:
# Histórico PETR4
# df = pd.read_csv('C:/Users/adrianalves/Desktop/Desafio Quant/dados/PETR4 Dados Históricos.csv')
# df.to_excel('C:/Users/adrianalves/Desktop/Desafio Quant/dados/PETR4 Dados Históricos.xlsx')
df = pd.read_excel('C:/Users/adrianalves/Desktop/Desafio Quant/dados/PETR4 Dados Históricos.xlsx')
df = df.rename(columns={"Vol.":'Volume'})
# df['Último'] = df['Último'].str.replace(',','.')
# df['Abertura'] = df['Abertura'].str.replace(',','.')
# df['Máxima'] = df['Máxima'].str.replace(',','.')
# df['Mínima'] = df['Mínima'].str.replace(',','.')
df['Volume'] = df['Volume'].str.replace(',','.').str.replace('M','')
df = df.dropna()
df

,Unnamed: 0,Data,Último,Abertura,Máxima,Mínima,Volume,Var%
0,3467,2009-12-18,28.67,29.07,29.16,28.58,16.19,-0.0080
1,3466,2009-12-21,27.74,29.00,29.04,27.74,20.94,-0.0324
2,3465,2009-12-22,28.64,28.16,28.64,28.06,15.04,0.0324
3,3464,2009-12-23,28.92,28.77,28.97,28.49,11.03,0.0098
4,3463,2009-12-28,28.96,29.09,29.16,28.87,9.12,0.0014
...,...,...,...,...,...,...,...,...
3463,4,2023-12-12,30.97,31.40,31.44,30.72,30.37,-0.0081
3464,3,2023-12-13,31.42,31.09,31.46,31.03,38.99,0.0144
3465,2,2023-12-14,32.10,31.81,32.48,31.66,57.95,0.0217
3466,1,2023-12-15,32.17,32.06,32.53,31.81,56.30,0.0023


Indicadores Técnicos 

In [3]:
import pandas as pd
import numpy as np

# Supondo que df já está carregado com as colunas ['Último', 'Máxima', 'Mínima', 'Volume']

df['Último'] = pd.to_numeric(df['Último'], errors='coerce')
df['Máxima'] = pd.to_numeric(df['Máxima'], errors='coerce')
df['Mínima'] = pd.to_numeric(df['Mínima'], errors='coerce')
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')


#############
# 1. Média Móvel Simples de 10 dias (MA)
df['Média Móvel'] = df['Último'].rolling(window=10).mean()

############
# 2. Média Móvel Ponderada de 10 dias (WMA)
def WMA(values, window):
    weights = np.arange(1, window + 1)
    return np.dot(values[-window:], weights) / weights.sum()

df['Média Móvel Ponderada'] = df['Último'].rolling(window=10).apply(lambda x: WMA(x, 10), raw=False)

######### 
# 3. Momentum (diferença em relação ao fechamento de 10 dias atrás)
df['Momentum'] = df['Último'].diff(periods=10)

###################### 
# 4. Oscilador Estocástico %K e %D (14 dias)
df['Lowest_14'] = df['Mínima'].rolling(window=14).min()
df['Highest_14'] = df['Máxima'].rolling(window=14).max()
df['%K'] = (df['Último'] - df['Lowest_14']) / (df['Highest_14'] - df['Lowest_14']) * 100
df['%D'] = df['%K'].rolling(window=3).mean()  # %D é a média de 3 dias de %K

################
# 5. Índice de Força Relativa (RSI) - 14 dias
def RSI(df, window=14):
    delta = df['Último'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

df['RSI'] = RSI(df)

######### 
# 6. Convergência e Divergência de Média Móvel (MACD)
df['EMA_12'] = df['Último'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['Último'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

################
# 7. Larry Williams %R (14 dias)
df['Williams_%R'] = (df['Highest_14'] - df['Último']) / (df['Highest_14'] - df['Lowest_14']) * -100

#################
# 8. Oscilador de Acumulação/Distribuição (A/D)
df['Money_Flow_Multiplier'] = ((df['Último'] - df['Mínima']) - (df['Máxima'] - df['Último'])) / (df['Máxima'] - df['Mínima'])
df['Money_Flow_Volume'] = df['Money_Flow_Multiplier'] * df['Volume']
df['A/D'] = df['Money_Flow_Volume'].cumsum()

################
# 9. Commodity Channel Index (CCI) - 20 dias
typical_price = (df['Último'] + df['Máxima'] + df['Mínima']) / 3
moving_average = typical_price.rolling(window=20).mean()
mean_deviation = typical_price.rolling(window=20).apply(lambda x: np.mean(np.abs(x - x.mean())))
df['CCI'] = (typical_price - moving_average) / (0.015 * mean_deviation)

# 10. PPO (Oscilador de Preço Percentual)
df['PPO'] = (df['EMA_12'] - df['EMA_26']) / df['EMA_26'] * 100



In [4]:
df = df.dropna()
columns_to_keep = ['Último', 'Máxima', 'Mínima', 'Volume', 'Média Móvel', 'Média Móvel Ponderada', 
                   'Momentum', '%K', '%D', 'RSI', 'MACD', 'Signal_Line', 'Williams_%R', 'A/D', 'CCI', 'PPO']

df = df[columns_to_keep].copy()
df

,Último,Máxima,Mínima,Volume,Média Móvel,Média Móvel Ponderada,Momentum,%K,%D,RSI,MACD,Signal_Line,Williams_%R,A/D,CCI,PPO
19,27.94,28.49,27.79,18.80,28.640,28.473818,-1.61,8.522727,34.892191,37.440758,-0.099522,-0.003569,-91.477273,-12.494141,-144.235310,-0.346832
20,27.06,28.14,27.02,24.59,28.418,28.186545,-2.22,1.581028,17.897833,31.474104,-0.214607,-0.045776,-98.418972,-35.327712,-219.622642,-0.751068
21,27.39,27.43,26.84,19.06,28.245,27.999636,-1.73,20.295203,10.132986,29.218107,-0.276002,-0.091821,-79.704797,-18.852119,-215.238308,-0.968910
22,26.72,27.06,26.52,20.32,28.014,27.722364,-2.31,6.600660,9.492297,26.893939,-0.374406,-0.148338,-93.399340,-24.120267,-218.345245,-1.320421
23,26.88,27.02,26.41,19.93,27.836,27.516182,-1.78,15.161290,14.019051,23.564356,-0.434473,-0.205565,-84.838710,-13.338464,-171.590719,-1.538187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3463,30.97,31.44,30.72,30.37,31.475,31.202909,-1.38,34.686347,42.435424,40.789474,-0.030442,0.126686,-65.313653,-2939.120904,-101.016395,-0.096989
3464,31.42,31.46,31.03,38.99,31.415,31.192909,-0.60,51.291513,43.419434,45.312500,-0.023118,0.096725,-48.708487,-2907.384857,-57.675739,-0.073649
3465,32.10,32.48,31.66,57.95,31.361,31.317455,-0.54,76.383764,54.120541,50.393701,0.037128,0.084806,-23.616236,-2903.144613,55.428018,0.118086
3466,32.17,32.53,31.81,56.30,31.336,31.464545,-0.25,78.966790,68.880689,52.403846,0.089491,0.085743,-21.033210,-2903.144613,65.957659,0.284135


Indicadores Financeiros 

In [16]:
df1 = pd.read_excel('C:/Users/adrianalves/Desktop/Desafio Quant/dados/Indicadores_Financeiros.xlsx')
df1 = df1.drop('Lucro Liquido Disponivel aos acionistas',axis=1)
df1 = df1.drop('Receita de vendas',axis=1)
df1 = df1.drop('Número de Ações',axis=1)
df1 = df1.drop('Ativo Circulante',axis=1)
df1 = df1.drop('Passivo Circulante',axis=1)
df1 = df1.drop('Estoques',axis=1)
df1

,Semestre,MLL,LA,ILS,PIB
0,2010-03-01,0.108431,0.445841,5.470533,9.21
1,2010-06-01,0.163019,0.631454,0.728438,8.52
2,2010-09-01,0.040729,1.598593,0.731388,6.91
3,2010-12-01,0.164994,2.697613,1.250920,5.69
4,2011-03-01,0.200454,0.842115,1.587176,5.20
5,2011-06-01,0.178024,0.838893,1.541274,4.70
6,2011-09-01,0.027835,0.484189,1.078872,3.54
7,2011-12-01,0.136430,2.553797,1.359248,2.57
8,2012-03-01,0.139323,0.706352,1.386998,1.71
9,2012-06-01,0.060493,0.622255,1.185856,0.99
